In [1]:
# default_exp quantification

# Quantification

> Functions related to quantification

This notebook contains everything to perform quantification

Current ToDo here:

- Most of the functions are not very well described yet
- Introductory text to give an overview / relevant papers would be nice.

In [2]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import string
import numpy as np
from itertools import combinations

from numba import njit
from numba.typed import List

from scipy.optimize import minimize
import random

from tqdm import tqdm as tqdm


def to_numba_list(a_list):
    numba_list = List()

    for element in a_list:
        numba_list.append(element)

    return numba_list


@njit
def fast_error(normalization, profiles):

    normalization = normalization.reshape(profiles[0].shape)

    total_error = 0
    for profile in profiles:
        total_error += fast_peptide_error(profile, normalization)

    return total_error


@njit
def fast_error_scalar(normalization, profiles):

    normalization = normalization.reshape(profiles[0].shape)

    total_error = np.zeros(len(profiles))

    for index in range(len(profiles)):
        profile = profiles[index]
        total_error[index] = fast_peptide_error(profile, normalization)

    return total_error


@njit
def fast_peptide_error(profile, normalization):

    pep_ints = np.zeros(profile.shape[1])

    normalized_profile = profile / normalization

    for i in range(len(pep_ints)):
        pep_ints[i] = np.nansum(normalized_profile[:, i])

    pep_ints = pep_ints[pep_ints > 0]

    # Loop through all combinations

    n = len(pep_ints)

    error = 0
    for i in range(n):
        for j in range(i + 1, n):
            error += np.abs(np.log(pep_ints[i] / pep_ints[j])) ** 2

    return error


def peptide_error(profile, normalization):
    """
    Calculates the peptide error for all runs

    """

    # Reshape normalization

    normalization = normalization.reshape(profiles[0].shape)

    pep_ints = np.nansum(profile / normalization, axis=0)

    pep_ints = pep_ints[pep_ints > 0]

    error = 0
    column_combinations = list(combinations(range(len(pep_ints)), 2))

    for ind1, ind2 in column_combinations:
        error += np.abs(np.log(pep_ints[ind1] / pep_ints[ind2])) ** 2

    return error


def elution_profile(timepoint, sigma, n_runs):
    return gaussian(timepoint, sigma, np.arange(0, n_runs))


def gaussian(mu, sigma, grid):
    norm = 0.3989422804014327 / sigma
    return norm * np.exp(-0.5 * ((grid - mu) / sigma) ** 2)


def return_elution_profile(timepoint, sigma, n_runs):
    return gaussian(timepoint, sigma, np.arange(0, n_runs))


def simulate_sample_profiles(n_peptides, n_runs, n_samples, threshold, use_noise=True):
    """
    Generate random profiles to serve as test data

    """

    abundances = np.random.rand(n_peptides)
    true_normalization = np.random.rand(n_runs, n_samples)

    # Question: Should we normalize for each run?

    maxvals = np.max(true_normalization, axis=1)
    #true_normalization / maxvals.reshape(len(maxvals), 1)

    elution_timepoints = random.choices(list(range(n_runs)), k=n_peptides)

    profiles = []

    for i in tqdm(range(n_peptides)):

        elution_timepoint = elution_timepoints[i]
        abundance = abundances[i]

        profile = elution_profile(elution_timepoint, 1, n_runs) * abundance
        elution_profiles = np.tile(profile, (n_samples, 1)).T

        # Some gaussian noise
        if use_noise:
            noise = np.random.normal(1, 0.2, elution_profiles.shape)
            noisy_profile = noise * elution_profiles
        else:
            noisy_profile = elution_profiles

        normalized_profile = noisy_profile * true_normalization

        normalized_profile[normalized_profile < threshold] = 0

        normalized_profile[normalized_profile == 0] = np.nan

        profiles.append(normalized_profile)

    return profiles, true_normalization


def simulate_peptide_intensities(
    n_sequences,
    n_samples,
    noise=True,
    remove=True,
    peptide_ratio=True,
    abundance=True,
    signal_level=100,
    noise_divider=10,
    keep=0.7,
):
    """
    Function simulate the peptide intensities

    Args:
        n_sequences: number of petpides to be simulated
        n_samples: number of samples
        noise: boolean to set whether to simulate noise or not
        remove: boolean to set whether to remove intensities below a certian threshold
        peptide_ratio: boolean to generate a random peptide ratio between samples, otherwise one
        abundance: boolean to simulate random abundance of peptides, otherwise one,
        signal_level: signal_level
        noise_divider: noise level
        keep: float for keep

    Returns:

    Raises:
        TODO: Include Exceptions
        TODO: Consider maximum size limit of npz files.
        TODO: also save a yaml file for this library
    """
    species = ["P" + str(_) for _ in range(1, n_sequences + 1)]
    sample = [string.ascii_uppercase[_] for _ in range(n_samples)]

    if peptide_ratio:
        peptide_ratio = np.random.rand(n_sequences)
        peptide_ratio = peptide_ratio / np.sum(peptide_ratio)
    else:
        peptide_ratio = np.ones(n_sequences)

    if abundance:
        ab_profile = np.random.rand(n_samples, 1)
    else:
        ab_profile = np.ones((n_samples, 1))

    original_signal = np.ones((n_samples, n_sequences))

    noise_sim = (np.random.rand(n_samples, n_sequences) - 0.5) / noise_divider

    if noise:
        noisy_signal = original_signal + noise_sim
        noisy_signal = noisy_signal * signal_level * peptide_ratio * ab_profile
    else:
        noisy_signal = (
            original_signal * signal_level * peptide_ratio * ab_profile
        )

    if remove:
        # Remove points
        keep_probability = keep
        to_remove = np.random.rand(n_samples, n_sequences)
        to_remove = to_remove >= keep_probability

        dummy_data = noisy_signal.copy()

        dummy_data[to_remove] = 0

    else:
        dummy_data = noisy_signal

    ground_truth = ab_profile.flatten()

    return dummy_data, ground_truth, species, sample


def get_protein_ratios(signal, minimum_ratios=2):
    """
    Function to calculate pairwise protein ratios
    """
    n_samples = signal.shape[0]

    column_combinations = list(combinations(range(n_samples), 2))
    ratios = np.empty((n_samples, n_samples))
    weights = np.zeros((n_samples, n_samples))
    ratios.fill(np.nan)

    for element in column_combinations:
        i = element[0]
        j = element[1]

        with np.errstate(divide="ignore", invalid="ignore"):
            ratio = np.true_divide(signal[j, :], signal[i, :])
            ratio[ratio == np.inf] = np.nan
            ratio[ratio == 0] = np.nan
            non_nan = np.sum(~np.isnan(ratio))

            if non_nan >= minimum_ratios:
                ratio = np.nanmedian(ratio)
            else:
                ratio = np.nan

        ratios[j, i] = ratio
        weights[j, i] = non_nan

    return ratios, weights


def triangle_error(normalization, ratios):
    """
    Cost function for peptide normalization
    For non-constrained solvers, returns scalar
    """
    normalization = np.abs(normalization)

    int_matrix = (
        np.repeat(normalization, len(normalization))
        .reshape((len(normalization), len(normalization)))
        .transpose()
    )
    with np.errstate(invalid="ignore"):
        x = (np.log(ratios) - np.log(int_matrix.T) + np.log(int_matrix)) ** 2

    return np.nansum(x)


def solve_BFGS(signal):
    """
    Wrapper to solve with BFGS
    BFGS is not constrained
    """

    ratios, weights = get_protein_ratios(signal)

    n_samples = ratios.shape[0]
    x0 = np.ones(n_samples)

    res_wrapped = minimize(
        triangle_error,
        args=ratios,
        x0=x0,
        method="BFGS",
        options={"maxiter": 2000, "disp": False},
    )
    if res_wrapped.success:
        solution = res_wrapped.x
        solution = np.abs(solution / np.max(solution))
    else:
        solution = x0
        print("Error. Minimization failed.")

    return solution


In [6]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_runner.ipynb.
Converted index.ipynb.
